# 한국어 파인 튜닝된 Neuron 모델을 SageMaker Endpoint INF2 에 배포하기

이 노트북은 [MLP-KTLim/llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 의 한국어 파인 튜닝 모델을 AWS Neuron Model 로 바꾼 [Gonsoo/AWS-HF-optimum-neuron-0-0-28-llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 모델을 사용 합니다.<br>
AWS Hugging Face Text Generation Inference (HF TGI) 도커 컨테이너를 이용하여, SageMaker Endpoint 에 배포 합니다.

### 커널 설정
- 쥬피터 노트북의 오른쪽 상단의 conda_neuron_pytorch_p38 를 사용합니다.

#### 코드 참조
- [Deploy Llama 3 70B on AWS Inferentia2 with Hugging Face Optimum](https://github.com/philschmid/huggingface-inferentia2-samples/blob/main/llama3-70b/deploy-llama-3-70b-inferentia2.ipynb)

---

## 1. 다음과 같은 패키지를 설치 합니다.



In [1]:
! pip install "sagemaker>=2.199.0" "gradio<4" transformers --upgrade --quiet
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
! pip list | grep -E "sagemaker|gradio|transformers|huggingface"

gradio                        3.50.2
gradio_client                 0.6.1
huggingface-hub               0.31.4
sagemaker                     2.243.2
sagemaker-core                1.0.29
sagemaker_pyspark             1.4.5
transformers                  4.46.3


## 2. SageMaker 세션 등 가져오기

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region}")


/home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/25/25 08:31:41] INFO     Found credentials from IAM Role:                                   ]8;id=378924;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=363351;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=789560;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=756380;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/25/25 08:31:42] INFO     Found credentials from IAM Role:                                   ]8;id=375803;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=481993;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=662699;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=22173;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker role arn: arn:aws:iam::057716757052:role/workshop-sagemaker-kfp-role2
sagemaker session region: us-west-2


## 3. Hugging Face LLM Inf2 DLC (Deep Learning Container ) 가져오기

새로운 Hugging Face TGI Neuronx DLC를 사용하여 AWS Inferentia2에서 추론을 실행할 수 있습니다. sagemaker SDK의 get_huggingface_llm_image_uri 메서드를 사용하여 원하는 backend, session, region, 그리고 version에 따라 적절한 Hugging Face TGI Neuronx DLC URI를 검색할 수 있습니다. 사용 가능한 모든 버전은 여기에서 확인할 수 있습니다.

**<u>"여기서는 2025.05.25" 현재 가장 최신인 SageMaker TGI 이미지를 사용합니다."</u>**

- 참조: [Hugging Face TGI Neuronx DLC URI](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true)



In [4]:
# TODO: Comment in when released
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
# llm_image = get_huggingface_llm_image_uri(
#   "huggingface-neuronx",
#   ## version="0.0.22"
# )

llm_image = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2"
print(f"llm image uri: \n {llm_image}")

llm image uri: 
 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2


## 4. 한국어 모델을 inferentia2 에 배포하기


모델을 Amazon SageMaker에 배포하기 전에 TGI Neuronx 엔드포인트 구성을 정의해야 합니다. 다음과 같은 추가 매개변수를 정의해야 합니다:

- HF_NUM_CORES: 컴파일에 사용된 Neuron 코어의 수.
- HF_BATCH_SIZE: 모델 컴파일에 사용된 배치 크기.
- HF_SEQUENCE_LENGTH: 모델 컴파일에 사용된 시퀀스 길이.
- HF_AUTO_CAST_TYPE: 모델 컴파일에 사용된 자동 캐스트 유형.

또한 기존의 TGI 매개변수도 정의해야 합니다:

- HF_MODEL_ID: Hugging Face 모델 ID.
- MAX_BATCH_SIZE: 모델이 처리할 수 있는 최대 배치 크기로, 컴파일에 사용된 배치 크기와 동일합니다.
- MAX_INPUT_LENGTH: 모델이 처리할 수 있는 최대 입력 길이.
- MAX_TOTAL_TOKENS: 모델이 생성할 수 있는 최대 총 토큰 수로, 컴파일에 사용된 시퀀스 길이와 동일합니다.

### 모델 배포 파라미터 설정 및 SageMaker HuggingFaceModel 생성

In [5]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.xlarge"
health_check_timeout=900 # additional time to load the model
volume_size=64 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "Gonsoo/AWS-HF-optimum-neuron-0-0-28-llama-3-Korean-Bllossom-8B",  
    "HF_NUM_CORES": "2", # number of neuron cores
    "HF_BATCH_SIZE": "4", # batch size used to compile the model
    "HF_SEQUENCE_LENGTH": "4096", # length used to compile the model
    "HF_AUTO_CAST_TYPE": "fp16",  # dtype of the model
    "MAX_BATCH_SIZE": "4", # max batch size for the model
    "MAX_INPUT_LENGTH": "2048", # max length of input text
    "MAX_TOTAL_TOKENS": "4096", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

                    INFO     Found credentials from IAM Role:                                   ]8;id=471949;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=69850;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### 모델을 SageMaker Endpoint 에 배포

HuggingFaceModel을 생성한 후에는 deploy 메서드를 사용하여 Amazon SageMaker에 배포할 수 있습니다. 우리는 ml.inf2.xlarge 인스턴스 유형을 사용하여 모델을 배포할 것입니다. TGI는 자동으로 모든 Inferentia 디바이스에 걸쳐 모델을 분산하고 샤딩할 것입니다.
- 다음은 약 17분이 소요 됩니다.

In [ ]:
%%time

# deactivate warning since model is compiled
llm_model._is_compiled_model = True

from datetime import datetime

# endpoint_name 생성 (sm-llama3-kr-inf2-yyyy-mm-dd-hh-mm-ss 형식)
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
endpoint_name = f"sm-llama3-kr-inf2-{timestamp}"


llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size,
  endpoint_name=endpoint_name
)

[05/25/25 08:31:43] INFO     Creating model with name:                                              ]8;id=316366;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=908145;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-05-25-08-31-43-396                       

                    INFO     Creating endpoint-config with name                                     ]8;id=765280;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=971648;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sm-llama3-kr-inf2-2025-05-25-08-31-42                                                 

[05/25/25 08:31:44] INFO     Creating endpoint with name sm-llama3-kr-inf2-2025-05-25-08-31-42      ]8;id=103238;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=684874;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4841\4841]8;;\

---------------------------!CPU times: user 461 ms, sys: 26 ms, total: 487 ms
Wall time: 14min 34s


## 5. 추론

엔드포인트가 배포된 후에는 이를 통해 추론을 실행할 수 있습니다. 우리는 predictor의 predict 메서드를 사용하여 엔드포인트에서 추론을 실행할 것입니다. 생성에 영향을 미치는 다양한 매개변수로 추론을 수행할 수 있습니다. 매개변수는 페이로드의 parameters 속성에서 정의할 수 있습니다. 지원되는 매개변수는 여기에서 확인할 수 있습니다.
메시지 API를 사용하면 대화 방식으로 모델과 상호작용할 수 있습니다. 메시지의 역할과 내용을 정의할 수 있습니다. 역할은 system, assistant 또는 user일 수 있습니다. system 역할은 모델에 컨텍스트를 제공하는 데 사용되고, user 역할은 질문을 하거나 모델에 입력을 제공하는 데 사용됩니다.

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is deep learning?" }
  ]
}
```

## 5.1. Message API 로 추론

In [ ]:
# Prompt to generate
messages=[
    { "role": "system", "content": "당신은 인공지능 전문가 입니다." },
    { "role": "user", "content": "딥러닝이 무엇인지 말해 주세요?" }
  ]

# Generation arguments
parameters = {
    "model": "meta-llama/Meta-Llama-3-70B-Instruct", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 2048,
    "stop": ["<|eot_id|>"],
}

이제 테스트 해보시죠

In [ ]:
chat = llm.predict({"messages" :messages, **parameters})
# chat = llm.predict({"messages" :messages, **parameters,"stream":True})

print(chat["choices"][0]["message"]["content"].strip())

딥러닝(Deep Learning)은 인공지능(AI)의 한 분야로, 인공신경망(Artificial Neural Networks)을 이용하여 데이터를 학습하고 예측하거나 분류하는 기술을 의미합니다. 딥러닝은 인간의 뇌 구조를 모방하여 여러 층의 신경망을 사용하여 복잡한 패턴을 학습하고 이해하는 데 중점을 둡니다.

딥러닝은 다음과 같은 특징을 가지고 있습니다:

1. **복잡한 데이터 처리**: 딥러닝은 고차원 데이터를 처리할 수 있으며, 이미지, 음성, 텍스트 등 다양한 형태의 데이터를 분석할 수 있습니다.
2. **자동 학습**: 딥러닝 모델은 데이터를 통해 자동으로 가중치와 편향을 조정하여 학습할 수 있습니다.
3. **다층 구조**: 여러 층의 신경망을 사용하여 복잡한 추상화와 패턴 인식을 가능하게 합니다.
4. **강력한 예측 능력**: 딥러닝 모델은 대규모 데이터를 학습함으로써 매우 높은 정확도의 예측을 할 수 있습니다.

딥러닝의 응용 분야는 다양합니다. 예를 들어, 이미지 인식, 음성 인식, 자연어 처리, 자율 주행, 의료 진단 등이 있습니다. 이러한 분야에서 딥러닝은 인간의 작업을 보조하거나 자동화하는 데 중요한 역할을 하고 있습니다.<|eot_id|>


## 5.2. Streaming 추론

In [ ]:
import boto3
import json

runtime = boto3.client('sagemaker-runtime')

payload = {
    "messages": messages,
    **parameters,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

# 수정된 스트리밍 응답 처리
for event in response['Body']:
    try:
        # Bytes를 string으로 decode
        chunk_bytes = event['PayloadPart']['Bytes']
        chunk_str = chunk_bytes.decode('utf-8')
        
        # 빈 문자열이나 공백만 있는 경우 건너뛰기
        if not chunk_str.strip():
            continue
            
        # SSE 형태 데이터 처리 (data: 접두사 제거)
        if chunk_str.startswith('data: '):
            chunk_str = chunk_str[6:]
            
        # [DONE] 메시지는 스트림 종료 신호
        if chunk_str.strip() == '[DONE]':
            break
            
        # JSON 파싱
        chunk = json.loads(chunk_str)
        
        # OpenAI 호환 형태 처리
        if 'choices' in chunk:
            delta = chunk['choices'][0].get('delta', {})
            if 'content' in delta:
                print(delta['content'], end='', flush=True)
        
        # TGI 원본 형태 처리 (token 기반)
        elif 'token' in chunk:
            if not chunk.get('finished', False):
                print(chunk['token']['text'], end='', flush=True)
                
    except json.JSONDecodeError as e:
        # JSON 파싱 실패 시 건너뛰기 (또는 디버깅용 출력)
        # print(f"JSON 파싱 에러: {chunk_str[:100]}")  # 디버깅용
        continue
    except KeyError:
        # 예상하지 못한 키 구조는 건너뛰기
        continue
    except Exception as e:
        # 기타 예외는 건너뛰기
        # print(f"기타 에러: {e}")  # 디버깅용
        continue

print()  # 마지막에 줄바꿈

[05/25/25 08:46:31] INFO     Found credentials from IAM Role:                                   ]8;id=436943;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=170105;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

딥러닝(Deep Learning)은 인공지능(AI)의 한 분야로, 인공 신경망(Artificial Neural Network)을 사용하여 데이터를 학습하고 예측하거나 결정을 내리는 기술입니다. 딥러닝은 인공 신경망의 다층 구조를 활용하여 복잡한 패턴을 학습하고, 자연어 처리, 이미지 인식, 음성 인식 등 다양한 분야에서 높은 성능을 보입니다.

딥러닝의 핵심 개념은 다음과 같습니다:

1. **인공 신경망 (Artificial Neural Network)**: 인공 신경망은 인간의 뇌를 모방한 구조로, 여러 층의 신경망(층)을 통해 입력 데이터를 처리하고 출력을 생성합니다. 각 층은 서로 연결된 뉴런(노드)으로 구성되어 있으며, 뉴런들은 활성화 함수를 통해 입력된 신호를 처리하여 출력을 생성합니다.

2. **다층 신경망 (Deep Neural Network)**: 다층 신경망은 여러 층의 인공 신경망을 쌓아 올린 구조로, 각 층은 이전 층의 출력을 입력으로 받아 더 복잡한 패턴을 학습합니다. 이러한 다층 구조 덕분에 딥러닝은 고차원적인 데이터를 처리하고, 복잡한 관계를 인식할 수 있습니다.

3. **데이터 학습 (Training)**: 딥러닝 모델은 대량의 데이터를 학습하여 모델의 가중치(Weight)를 조정합니다. 학습 과정에서 모델은 오차를 최소화하기 위해 데이터를 반복적으로 처리하며, 이를 통해 최적의 모델을 찾습니다.

4. **오버피팅 (Overfitting)과 언더피팅 (Underfitting)**: 딥러닝 모델이 학습 데이터보다 복잡한 패턴을 학습하는 오버피팅과, 학습 데이터의 간단한 패턴을 잘 학습하지 못하는 언더피팅은 모델의 성능에 큰 영향을 미칠 수 있습니다. 이를 방지하기 위해 정규화, 드롭아웃, 레이어 드롭 등 다양한 기법이 사용됩니다.

딥러닝은 자연어 처리, 이미지 인식, 음성 인식, 자율 주행, 의료 진단 등 다양한 분야에서 혁신적인 성과를 내고 있으며, 계속해서 발전하고 있습니다.


## 6. 리소스 정리

SageMaker HuggingFace Model 및 엔드포인트를 삭제 합니다.


In [ ]:
llm.delete_endpoint()

[05/25/25 08:46:53] INFO     Deleting endpoint configuration with name:                             ]8;id=813182;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=66164;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4995\4995]8;;\
                             sm-llama3-kr-inf2-2025-05-25-08-31-42                                                 

                    INFO     Deleting endpoint with name: sm-llama3-kr-inf2-2025-05-25-08-31-42     ]8;id=705683;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=45825;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4985\4985]8;;\